In [241]:
import numpy as np
import pandas as pd
from datetime import datetime
import calendar
from datetime import timedelta
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [242]:
from datetime import datetime
import calendar
from datetime import timedelta
import datetime as dt

In [262]:
# train data consists of 55M rows, taking 4M rows initially
train = pd.read_csv("./iiitb2019nyctaxifare/train.csv/train.csv", nrows = 4000000)
#test = pd.read_csv("./iiitb2019nyctaxifare/test.csv/test.csv")

In [263]:
train.head()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000000 entries, 0 to 3999999
Data columns (total 8 columns):
key                  object
fare_amount          object
pickup_datetime      object
pickup_longitude     object
pickup_latitude      object
dropoff_longitude    object
dropoff_latitude     object
passenger_count      object
dtypes: object(8)
memory usage: 244.1+ MB


In [245]:
test.head()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11084772 entries, 0 to 11084771
Data columns (total 7 columns):
key                  object
pickup_datetime      object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count      int64
dtypes: float64(4), int64(1), object(2)
memory usage: 592.0+ MB


In [246]:
# change the format of datetime
def change_datetime_format(data):
    data['pickup_datetime']=pd.to_datetime(data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S UTC')
    return data

In [247]:
# split the datetime
def split_datetime(data):
    data['pickup_date']= data['pickup_datetime'].dt.date
    data['pickup_day']=data['pickup_datetime'].apply(lambda x:x.day)
    data['pickup_hour']=data['pickup_datetime'].apply(lambda x:x.hour)
    data['pickup_day_of_week']=data['pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
    data['pickup_month']=data['pickup_datetime'].apply(lambda x:x.month)
    data['pickup_year']=data['pickup_datetime'].apply(lambda x:x.year)
    return data

In [266]:
# removing outliers in latitude and longitude
def remove_outliers(data):
    
    misplaced_locations_index = []

    for i, val in enumerate(zip(data.pickup_latitude,data.dropoff_latitude,data.pickup_longitude,data.dropoff_longitude)):

        #print(val)
        #break

        lat1,lat2,lon1,lon2 = val
        #co_ords1 = (lat1, lon1)
        #co_ords2 = (lat2, lon2)

        if float(lat1) < 40.5 or float(lat1) > 41.8 or float(lat2) < 40.5 or float(lat2) > 41.8 or float(lon1) < -74.5 or float(lon1) > -72.8 or float(lon2) < -74.5 or float(lon2) > -72.8:
            misplaced_locations_index.append(i)


    data = data.drop(misplaced_locations_index)
    #data = data.replace(misplaced_locations_index, 50)
    return data

In [249]:
def remove_null(data):
    data = data.dropna()
    #data = data.fillna(1)
    return data

In [250]:
def remove_neg_pass_count(data):
    data = data.drop(data[data['passenger_count'] <= 0].index.tolist())
    #data = data.replace(data[data['passenger_count'] <= 0].index.tolist(), 1)
    return data

In [251]:
def remove_neg_fare_amount(data):
    data = data.drop(data[data['fare_amount'] <= 0].index.tolist())
    #data = data.replace(data[data['fare_amount'] <= 0].index.tolist(), 1)
    return data

In [252]:
#calculate trip distance in miles
def distance(lat1, lat2, lon1,lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a))

In [253]:
def calc_distance(data):
    data['trip_distance']=data.apply(lambda row:distance(row['pickup_latitude'],row['dropoff_latitude'],row['pickup_longitude'],row['dropoff_longitude']),axis=1)
    return data

In [254]:
# Let us encode day of the week to numbers
def encodeDays(day_of_week):
    day_dict={'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6}
    return day_dict[day_of_week]

In [255]:
def pickupday_encode(data):
    data['pickup_day_of_week']=data['pickup_day_of_week'].apply(lambda x:encodeDays(x))
    return data

In [256]:
def drop_columns(data):
    data.drop(columns=['key','pickup_datetime','pickup_date'], inplace=True)
    return data

In [267]:
# clean the data
def cleandata(data):
    data = remove_outliers(data)
    data = remove_null(data)
    data = change_datetime_format(data)
    data = split_datetime(data)
    data = remove_neg_pass_count(data)
    data = remove_neg_fare_amount(data)
    data = calc_distance(data)
    data = pickupday_encode(data)
    data = drop_columns(data)
    
    return data

In [268]:
train = cleandata(train)

ValueError: could not convert string to float: 'pickup_latitude'

In [158]:
train.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_day,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,trip_distance
0,4.5,-73.986910,40.739538,-73.991381,40.745614,2,20,23,3,10,2010,0.480646
1,4.1,-73.961572,40.760283,-73.957438,40.769387,5,30,10,3,12,2009,0.665188
2,6.1,-73.979437,40.746517,-73.984195,40.732117,1,20,11,5,7,2012,1.025650
3,4.9,-73.964097,40.792508,-73.976422,40.785767,1,31,11,2,5,2011,0.795377
4,6.5,-74.003943,40.725670,-73.988915,40.748370,1,25,17,2,5,2010,1.754687


In [159]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390181 entries, 0 to 399999
Data columns (total 12 columns):
fare_amount           390181 non-null float64
pickup_longitude      390181 non-null float64
pickup_latitude       390181 non-null float64
dropoff_longitude     390181 non-null float64
dropoff_latitude      390181 non-null float64
passenger_count       390181 non-null int64
pickup_day            390181 non-null int64
pickup_hour           390181 non-null int64
pickup_day_of_week    390181 non-null int64
pickup_month          390181 non-null int64
pickup_year           390181 non-null int64
trip_distance         390181 non-null float64
dtypes: float64(6), int64(6)
memory usage: 38.7 MB


In [160]:
#test = cleandata(test)

In [185]:
test.head()

,key,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_day,pickup_hour,pickup_day_of_week,pickup_month,pickup_year
0,2009-01-01 00:01:04.0000003,-73.972484,40.742743,-73.918937,40.764496,1,1,0,4,1,2009
1,2009-01-01 00:01:26.0000001,-73.985850,40.722826,-73.986301,40.739347,1,1,0,4,1,2009
2,2009-01-01 00:04:42.0000001,-73.988917,40.740142,-73.982769,40.777291,1,1,0,4,1,2009
3,2009-01-01 00:04:54.0000001,-73.977163,40.764490,-73.914474,40.771575,1,1,0,4,1,2009
4,2009-01-01 00:04:59.0000004,-73.948849,40.778003,-73.977678,40.748692,2,1,0,4,1,2009


In [162]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11084772 entries, 0 to 11084771
Data columns (total 7 columns):
key                  object
pickup_datetime      object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count      int64
dtypes: float64(4), int64(1), object(2)
memory usage: 592.0+ MB


In [163]:
print("Shape of Training Data after cleaning ",train.shape)
print("Shape of Testing Data after cleaning", test.shape)

Shape of Training Data after cleaning  (390181, 12)
Shape of Testing Data after cleaning (11084772, 7)


In [164]:
def processDataForModelling(data,target,is_train=True,split=0.25):
    data_1=data
    # One hot Encoding
    data_1=pd.get_dummies(data_1)
    if is_train==True:
        X=data_1.drop([target],axis=1)
        y=data_1[target]
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=split,random_state=123)
        
        print("Shape of Training Features",X_train.shape)
        print("Shape of Validation Features ",X_test.shape)
        
        return X_train, X_test, y_train, y_test
    else:
        print ("Shape of Test Data",data_1.shape)
        return data_1

In [165]:
X_train, X_test, y_train, y_test=processDataForModelling(train,'fare_amount',is_train=True,split=0.2)

Shape of Training Features (312144, 11)
Shape of Validation Features  (78037, 11)


In [126]:
# Baseline Model
avg_fare=round(np.mean(y_train),2)

baseline_pred=np.repeat(avg_fare, y_test.shape[0])
#baseline_rmse=np.sqrt(mean_squared_error(baseline_pred, y_test))
#print("Basline RMSE of Validation data :",baseline_rmse)

In [166]:
# Linear Regression Model
lm = LinearRegression()
lm.fit(X_train,y_train)
y_pred=np.round(lm.predict(X_test),2)
lm_rmse=np.sqrt(mean_squared_error(y_pred, y_test))
print("RMSE for Linear Regression is ",lm_rmse)

RMSE for Linear Regression is  5.410930065232967


In [54]:
#test_data=processDataForModelling(test,'fare_amount',is_train=False)

Shape of Test Data (10811259, 11)


In [167]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11084772 entries, 0 to 11084771
Data columns (total 7 columns):
key                  object
pickup_datetime      object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count      int64
dtypes: float64(4), int64(1), object(2)
memory usage: 592.0+ MB


In [128]:
#avg_fare = 11.33
#baseline_pred=np.repeat(avg_fare,test.shape[0])

In [129]:
#baseline_pred


array([11.33, 11.33, 11.33, ..., 11.33, 11.33, 11.33])

In [130]:
#pred = pd.DataFrame(baseline_pred)

In [131]:
#pred.info(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11084772 entries, 0 to 11084771
Data columns (total 1 columns):
0    float64
dtypes: float64(1)
memory usage: 84.6 MB


In [132]:
#datasets = pd.concat([test['key'],pred[0]], axis = 1)

In [133]:
#datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11084772 entries, 0 to 11084771
Data columns (total 2 columns):
key    object
0      float64
dtypes: float64(1), object(1)
memory usage: 169.1+ MB


In [134]:
#datasets.columns = ['key','fare_amount']

In [111]:
#datasets = datasets.dropna()

In [135]:
#datasets.isnull().sum(axis = 0)

key            0
fare_amount    0
dtype: int64

In [136]:
#datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11084772 entries, 0 to 11084771
Data columns (total 2 columns):
key            object
fare_amount    float64
dtypes: float64(1), object(1)
memory usage: 169.1+ MB


In [137]:
#datasets.to_csv('sub1.csv',index=False)

In [138]:
#d1 = pd.read_csv('sub1.csv')

In [139]:
#d1.shape

(11084772, 2)

In [169]:
#d1.head(140)

In [239]:
#test = cleandata(test)

In [174]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11084772 entries, 0 to 11084771
Data columns (total 11 columns):
key                   object
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
passenger_count       int64
pickup_day            int64
pickup_hour           int64
pickup_day_of_week    int64
pickup_month          int64
pickup_year           int64
dtypes: float64(4), int64(6), object(1)
memory usage: 930.3+ MB


In [219]:
test_data.shape

(10811259, 11)

In [220]:
## Now Linear regression on test
lr_pred=np.round(lm.predict(test_data),2)

In [222]:
lr_pred

array([13.46,  7.01, 10.66, ..., 12.45, 30.46, 11.49])

In [223]:
test_data.columns

Index(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'passenger_count', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year', 'trip_distance'],
      dtype='object')

In [225]:
test.shape

(11084772, 13)

In [226]:
pred_lr = pd.DataFrame(lr_pred)

In [227]:
datasets_lr = pd.concat([test['key'],pred_lr[0]], axis = 1)

In [228]:
datasets_lr.shape

(11084772, 2)

In [229]:
datasets_lr.head

<bound method NDFrame.head of                                   key      0
0         2009-01-01 00:01:04.0000003  13.46
1         2009-01-01 00:01:26.0000001   7.01
2         2009-01-01 00:04:42.0000001  10.66
3         2009-01-01 00:04:54.0000001  13.24
4         2009-01-01 00:04:59.0000004  10.42
...                               ...    ...
11084767  2015-06-30 23:58:39.0000006    NaN
11084768  2015-06-30 23:58:58.0000004    NaN
11084769  2015-06-30 23:59:26.0000003    NaN
11084770  2015-06-30 23:59:48.0000003    NaN
11084771  2015-06-30 23:59:54.0000003    NaN

[11084772 rows x 2 columns]>

In [230]:
datasets_lr.columns = ['key','fare_amount']

In [231]:
datasets_lr.isnull().sum(axis = 0)

key                 0
fare_amount    273513
dtype: int64

In [232]:
datasets_lr = datasets_lr.fillna(11.33)

In [234]:
datasets_lr.shape


(11084772, 2)

In [235]:
datasets_lr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11084772 entries, 0 to 11084771
Data columns (total 2 columns):
key            object
fare_amount    float64
dtypes: float64(1), object(1)
memory usage: 169.1+ MB


In [236]:
datasets_lr.isnull().sum(axis = 0)

key            0
fare_amount    0
dtype: int64

In [237]:
datasets_lr.to_csv('sub_lr1.csv',index=False)

In [238]:
d_lr1 = pd.read_csv('sub_lr1.csv')

In [240]:
d_lr1.shape

(11084772, 2)